In [1]:
import pandas as pd
import numpy as np
import time
import iddModel.learnUsingNumpy as idoctor
import multiprocessing
import parmap 
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

In [2]:
tuningSetAgeRate = [[0.1, 0.2, 0.9],[26,28,27],[40,40,40]]
tuningSetRetireRate = [1.4,35,0.2]
%time npCostData = idoctor.getCost([tuningSetAgeRate, tuningSetRetireRate])
pd.DataFrame(npCostData)

CPU times: user 17.9 ms, sys: 0 ns, total: 17.9 ms
Wall time: 15.4 ms


,0,1,2,3,4,5,6,7,8,9,10,11
0,9418,7496,7192,158,4060,1368,0.0233792,0.0756676,0.0462088,0.0397671,0.0729549,0.0325335
1,5614,4018,521,35,425,1535,0.0233792,0.0406015,0.0109752,0.0107433,0.00915858,0.0136547
2,9503,6008,6718,37,2715,471,0.0240397,0.0573803,0.0384127,0.0404125,0.0572331,0.022187
3,5861,3130,1027,174,151,1681,0.0240397,0.0300724,0.00643247,0.00824714,0.00615157,0.0147985
4,8552,4448,5359,298,1408,451,0.0216337,0.0396751,0.0253656,0.0371585,0.0393722,0.0101437
5,5480,2282,828,285,238,1847,0.0216337,0.0203801,0.00769297,0.00576102,0.00164442,0.016352
6,8634,3174,4666,357,275,1426,0.0229512,0.0251427,0.0169478,0.0369848,0.0276788,0.000378146
7,5841,1636,1091,479,609,2026,0.0229512,0.0127482,0.00563315,0.0037553,0.00191342,0.017678
8,8700,1681,4360,818,994,2483,0.0235096,0.00919986,0.0127802,0.0402312,0.0151617,0.00990313
9,6093,631,1590,651,1060,2161,0.0235096,0.00198843,0.000982141,0.00212644,0.0060674,0.0185624


In [3]:
def runGetCost(tuningValueList):
    resultData = []
    
    for value in tuningValueList:
        resultData.append([value ,idoctor.getCost([[[value[0], value[1], value[2]],[26,28,27],[40,40,40]], [value[3],30,value[4]]])])
        
        
    return resultData

In [33]:
def startGetCost():
    tAge1 = np.arange(0.05, 1, 0.05) # 연령분포모델1 변수 / 범위 0~1
    tAge2 = np.arange(0.05, 1, 0.05) # 연령분포모델2 변수 / 범위 0~1
    tAge3 = np.arange(0.1, 1, 0.1) # 연령분포모델3 변수 / 범위 0~1
    tRetire1 = np.arange(1.1, 3, 0.1) # 은퇴율모델 변수1 / 범위 1 이상
    tRetire2 = np.arange(0.1, 10, 0.1) # 은퇴율모델 변수3 / 범위 0 이상
    
    dataSize = len(tAge1)*len(tAge2)*len(tAge3)*len(tRetire1)*len(tRetire2)
    timeMinute = dataSize*20/1000/60/22
    curentData = 0
    resultData = []
    
    num_cores = 30

    now = time.localtime()
    print("│ Start Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    print("│ expect running time : " + str(round(timeMinute)) + "min")
    print("│ Data Size = " + str(dataSize))
    print("└───────────────────────")
    print('│ '+str(round(curentData/dataSize*100,2))+'%')
    
    tuningList = []
    
    for i in tAge1:
        for j in tAge2:
            for k in tAge3:
                for m in tRetire1:
                    for n in tRetire2:
                        tuningList.append([i,j,k,m,n])

    
    splitedTuningList = np.array_split(tuningList, num_cores) 
    splitedTuningList = [i.tolist() for i in splitedTuningList] 
    
    result = parmap.map(runGetCost, splitedTuningList, pm_pbar=True, pm_processes=num_cores)
    
    print("| End Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    return result

In [34]:
%time liCostData = startGetCost()
npCostData = np.array(liCostData)

np.shape(npCostData)

│ Start Time : 2020/11/10 18:24:52
│ expect running time : 93min
│ Data Size = 6111369
└───────────────────────
│ 0.0%


100%|██████████| 30/30 [1:39:23<00:00, 198.77s/it]   


| End Time : 2020/11/10 18:24:52
CPU times: user 2min 21s, sys: 1min 17s, total: 3min 38s
Wall time: 1h 39min 52s


(30,)

In [35]:
len(liCostData[0])
npCostData = np.array(liCostData)

In [ ]:
value = [0.4, 0.8, 0.4, 40, 1.09, 30, 1.9]

data = idoctor.getCost([[[value[0], value[1], value[2]],[26,28,27],[value[3],value[3],value[3]]], [value[4],value[5],value[6]]])
data

In [42]:
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minTuningValue = npCostData[0][0][0]
minTuningValue

[0.05, 0.05, 0.1, 1.1, 0.1]

In [51]:
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minManTuningValue = npCostData[0][0][0]
minWomanTuningValue = npCostData[0][0][0]
count = 0
for data1 in npCostData:
    for data2 in data1:   
        count += 1
        npManCost = (np.concatenate((data2[1][0],data2[1][2],data2[1][4],data2[1][6],data2[1][8],data2[1][10]))).reshape(6,12).sum(axis=0)
        npWomanCost = (np.concatenate((data2[1][1],data2[1][3],data2[1][5],data2[1][7],data2[1][9],data2[1][11]))).reshape(6,12).sum(axis=0)

        manCostCount = 0
        womanCostCount = 0
        
        for i in range(12):
            if npManCost[i]<=minManCost[i]:
                manCostCount += 1
            else:
                break
                
        for i in range(12):
            if npWomanCost[i]<=minWomanCost[i]:
                womanCostCount += 1
            else:
                break
                
        if manCostCount==12:
            minManCost = npManCost
            minManTuningValue = data2[0]
            
        if womanCostCount==12:
            minWomanCost = npWomanCost
            minWomanTuningValue = data2[0]

pd.DataFrame([[minManCost, minManTuningValue],[minWomanCost,minWomanTuningValue]])
print(count)

6111369


In [52]:
print(minManCost)
print(minManTuningValue)
print(minWomanCost)
print(minWomanTuningValue)

[49580.0 24945.0 32934.0 1822.0 11958.0 8281.0 0.13810982645232572
 0.2273820888936964 0.16591566654493078 0.21607463116044057
 0.23005667409844557 0.10861255367502023]
[0.05, 0.05, 0.6, 1.2000000000000002, 1.8000000000000003]
[34487.0 13402.0 7116.0 2220.0 3633.0 10344.0 0.14395946825812597
 0.12809288631690458 0.031208756300101795 0.03560435146605327
 0.03501055575021555 0.09011246067102151]
[0.05, 0.05, 0.1, 1.1, 0.1]
